In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
class DMoneyMarketSwap:    
    def __init__(self, evDate, effective, maturity, position, CCYComm, InitNPAComm, 
                 FinalNPAComm, CCYTerm, InitNPATerm, FinalNPATerm, FXSpot, h_discTSComm,
                 h_discTSTerm):
        self._ValueDate = evDate
        self._EffectiveDate = effective
        self._Maturity = maturity        
        if position == 'Buy':          
            self._Position = 1.0
        else:
            self._Position = -1.0
        self._CCYComm = CCYComm
        self._InitNPAComm = InitNPAComm
        self._FinalNPAComm = FinalNPAComm
        self._CCYTerm = CCYTerm
        self._InitNPATerm = InitNPATerm
        self._FinalNPATerm = FinalNPATerm
        self._FXSpot = FXSpot
        self._H_discTSComm = h_discTSComm
        self._H_discTSTerm = h_discTSTerm
        
    def NPV(self):
        FinalDfComm = self._H_discTSComm.discount(self._Maturity)
        FinalDfTerm = self._H_discTSTerm.discount(self._Maturity)
        FinalPVComm = FinalDfComm * self._FinalNPAComm
        FinalPVTerm = FinalDfTerm * self._FinalNPATerm        
        if (self._ValueDate >= self._EffectiveDate):        
            self._Value = self._Position * (FinalPVComm * self._FXSpot - FinalPVTerm)
        else:
            InitDfComm = self._H_discTSComm.discount(self._EffectiveDate)
            InitDfTerm = self._H_discTSTerm.discount(self._EffectiveDate)
            InitPVComm = InitDfComm * self._InitNPAComm
            InitPVTerm = InitDfTerm * self._InitNPATerm        
            self._Value = self._Position * ((FinalPVComm * self._FXSpot - FinalPVTerm) 
                                            + (InitPVTerm - InitPVComm * self._FXSpot))
        return self._Value

In [4]:
PstBook = xl.load_workbook('Pst_MMS.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
TermCCY = 'TWD'
Type = 'MMS'
CommCCY = MktSheet.cell(3, 5).value
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [5]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [6]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [7]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [8]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [9]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y])
ListDate
ListCommRate
ListTermRate

[-0.00392904297456824,
 -0.003929042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [10]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [11]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)

In [12]:
MMSSheet = PstBook['MMS']

In [13]:
start_row = 2
end_row = 4
start_col = 0
end_col = 10
recordcount = end_col - start_col + 1
ListMMS = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in MMSSheet[i][start_col:end_col+1]]
    ListMMS.append(row)
    print(row)

['MMS001', datetime.datetime(2021, 5, 19, 0, 0), datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Buy', 'USD', 1000000, 1000000, 'TWD', 28000000, 27900000]
['MMS002', datetime.datetime(2021, 6, 13, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Sell', 'USD', 1000000, 1000000, 'TWD', 27900000, 27800000]
['MMS003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Buy', 'USD', 1000000, 1000000, 'TWD', 27850000, 27700000]


In [14]:
MMSBook = list()
for i in range(len(ListMMS)):
    row = ListMMS[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    MMSBook.append(record)

In [15]:
MMSBook

[['MMS001',
  Date(19,5,2021),
  Date(21,5,2021),
  Date(21,9,2021),
  'Buy',
  'USD',
  1000000,
  1000000,
  'TWD',
  28000000,
  27900000],
 ['MMS002',
  Date(13,6,2021),
  Date(15,6,2021),
  Date(15,12,2021),
  'Sell',
  'USD',
  1000000,
  1000000,
  'TWD',
  27900000,
  27800000],
 ['MMS003',
  Date(21,6,2021),
  Date(23,6,2021),
  Date(23,5,2022),
  'Buy',
  'USD',
  1000000,
  1000000,
  'TWD',
  27850000,
  27700000]]

In [16]:
SenSheet = PstBook['FXR']
for i in range(0, len(MMSBook)):
    ListValue = list()
    record = MMSBook[i]
    tradeDate = record[1]
    effectiveDate = record[2]
    maturityDate = record[3]
    position = record[4]
    CommCCY = record[5]    
    InitCommAmt = record[6]
    FinalCommAmt = record[7]
    TermCCY = record[8]    
    InitTermAmt = record[9]
    FinalTermAmt = record[10]
    MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY, 
                           InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                           FinalTermAmt, FXSpot, h_TSCommZero, h_TSTermZero)    
    Value = MMS.NPV()
    print(Value)
    
    FXSpot_1 = FXSpot * 1.01
    MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY, 
                           InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                           FinalTermAmt, FXSpot_1, h_TSCommZero, h_TSTermZero)    
    Value_1 = MMS.NPV()
    Delta = (Value_1 - Value) * 100.0
    print(Value_1)
    
    FXSpot_Up = FXSpot * 1.15
    MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY, 
                           InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                           FinalTermAmt, FXSpot_Up, h_TSCommZero, h_TSTermZero)
    Value_Up = MMS.NPV()
    print(Value_Up)
    
    FXSpot_Dn = FXSpot * 0.85
    MMS = DMoneyMarketSwap(evDate, effectiveDate, maturityDate, position, CommCCY, 
                           InitCommAmt, FinalCommAmt, TermCCY, InitTermAmt,
                           FinalTermAmt, FXSpot_Dn, h_TSCommZero, h_TSTermZero)    
    Value_Dn = MMS.NPV()
    print(Value_Dn)
    
    Curv_Up = (-((Value_Up - Value) - 0.15 * Delta))
    Curv_Dn = (-((Value_Dn - Value) + 0.15 * Delta))    
    
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = Value
    SenSheet.cell(i+5, 3).value = TermCCY
    SenSheet.cell(i+5, 4).value = Delta
    SenSheet.cell(i+5, 6).value = Curv_Up
    SenSheet.cell(i+5, 8).value = Curv_Dn
    print(Delta)
    print(Curv_Up)
    print(Curv_Dn)
    
PstBook.save('Pst_MMS.xlsx')
PstBook.close()

-43323.58480441943
235492.56429441273
4138918.651678074
-4225565.821286913
27881614.909883216
-1.1175870895385742e-08
1.1175870895385742e-08
-50456.49452036247
-329113.6512955688
-4230313.846148457
4129400.857107736
-27865715.677520633
-0.0
-3.725290298461914e-09
149766.27802324295
428036.6884830296
4323822.434920043
-4024289.878873557
27827041.045978665
-0.0
-0.0
